In [ ]:
!pip install pymorphy2 
!pip install pymorphy2-dicts
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np 
import json 
import io

import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split

import pymorphy2
from tqdm import tqdm
from catboost import CatBoostClassifier, Pool

In [ ]:
with io.open("/content/drive/MyDrive/hacaton_ai2021spb/data3.json", mode="r", encoding="utf-8") as file:
    js = json.load(file)

In [ ]:
all_data = []
for k in js:
  data = js[k]
  company = data['company']
  gos = company['goszakupki']
  habr = company['habr']
  vcru = company['vcru']
  tj = company['tinkoff_journal']
  people = data['people']
  label = data['nomination'].replace('\n', '').replace('   ', '')

  texts_kw = habr["company_info"]["description"] if habr["company_info"] else '' + ' '  + ' '.join(habr["company_info"]["industries"] if habr["company_info"] else []) + ' '  + ' '.join(tj['texts']) + ' ' + ' '.join([x['text'] for x in company["cnews"]]) + ' '.join([x['description'] for x in gos] if gos else []) 

  texts_pos_neg = []
  for text in company['cnews']:
    texts_pos_neg.append(text['text'])
  for text in company['habr']['references']:
    texts_pos_neg.append(text)
  for text in tj['texts']:
    texts_pos_neg.append(text)
  for text in vcru['texts']:
    texts_pos_neg.append(text)

  people_texts = []
  for person in people:
    for text in person['cnews']:
      people_texts.append(text['text'])
    for text in person['habr']['references']:
      people_texts.append(text)

  all_data.append([k, texts_kw, texts_pos_neg, people_texts, label])


In [ ]:
df = pd.DataFrame(data=all_data, columns=['company_name', 'text_for_keywords', 'text_for_classification', 'people', 'label'])
df_company_kw = pd.concat([df['company_name'], df['text_for_keywords'], df['label']], axis=1)
encode_company_classes = {x: i for (i,x) in enumerate(np.unique(list(df['label'])))}

In [ ]:
cv = CountVectorizer()
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)

def get_kw(text, first=True):
  sw = '''что из есть он том они сейчас быть который время например поэтому несколько раз тут а е и ж м о на не ни об но он мне мои мож она они оно мной много многочисленное многочисленная многочисленные многочисленный мною мой мог могут можно может можхо мор моя моё мочь над нее оба нам нем  нами ними мимо немного одной одного менее однажды однако меня нему меньше ней наверху него ниже мало надо один одиннадцать одиннадцатый назад наиболее недавно миллионов недалеко между низко меля нельзя нибудь непрерывно наконец никогда никуда нас наш нет нею неё нихмира наша наше наши ничего начала нередко несколько обычно опять около мы ну нх от отовсюду особенно нужно очень отсюда в во вон вниз внизу вокруг вот восемнадцать восемнадцатый восемь восьмой вверх вам вами важное важнаяважные важный  вдали везде ведь вас ваш ваша ваше ваши впрочем весь вдруг вы все второй всем всеми времени время всему всего всегда всех всею всю вся всё всюду г год говорил говорит года году где да ееза из ли же им до по ими под иногда довольно именно долго позже более должно пожалуйста значит иметь больше пока ему имя пор пора потом потому после почему почти посреди ей два две двенадцать двенадцатый двадцать двадцатый двух его дел или без день занят занята занято заняты действительно давно девятнадцать девятнадцатый девять девятый даже алло жизнь далеко близко здесь дальше для лет зато даром первый перед затем зачем лишь десять десятый ею её их бы еще прибыл про  процентов против просто бывает бывь если люди была были было  будем  будет будете будешь прекрасно буду будь будто будут ещё пятнадцать пятнадцатый друго другое другой другие другая других есть пять быть лучше пятый к ком конечно кому кого когда которой которого которая которые который которых кем каждое каждая каждые каждый кажется как какой какая кто кроме  куда кругом с  т у я та те уж со то том снова тому совсем того тогда тоже собой тобой собою тобою сначалатолько уметь  тот тою хорошо хотеть хочешь хоть хотя свое свои твой своей своего своих свою твоя твоё раз уже сам там тем чем сама сами теми само рано самом самому самой самого семнадцать семнадцатый самим самими самихсаму семь  чему раньше сейчас чего сегодня себе тебе сеаой человек разве теперь себя тебя седьмой спасибо слишком так такое такой такие также такая сих тех чаще четвертый через  часто шестой шестнадцать шестнадцатый шесть четыре четырнадцать четырнадцатый сколько сказал сказала сказать ту ты три эта эти что это чтоб этом этому этой этого чтобы этот стал туда этим этимирядом тринадцать  тринадцатый этих третий тут эту суть чуть тысяч'''.split()
  def lemmatize(texts):
      m = pymorphy2.MorphAnalyzer() 
      texts = [m.parse(x)[0].normal_form for x in texts]
      return texts

  def pre_process(text, sw):
      
      text=text.lower()
      
      text=re.sub("</?.*?>"," <> ",text)
      
      text=re.sub("(\\d|\\W)+"," ",text)
      text_splitted = []
      for word in text.split():
        if word not in sw:
          text_splitted.append(word)
      lem = lemmatize(text_splitted)
      text_splitted_n = []
      for word in lem:
        if word not in sw:
          text_splitted_n.append(word)   
      return text_splitted_n

  text_n = pre_process(text, sw)
  if first:
    word_count_vector = cv.fit_transform(text_n)
    tfidf_transformer.fit(word_count_vector)
  else:
    word_count_vector = cv.transform(text_n)

  def sort_coo(coo_matrix):
      tuples = zip(coo_matrix.col, coo_matrix.data)
      return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

  def extract_topn_from_vector(feature_names, sorted_items, topn=10):
      
      sorted_items = sorted_items[:topn]

      score_vals = []
      feature_vals = []

      for idx, score in sorted_items:
          fname = feature_names[idx]
          
          score_vals.append(round(score, 3))
          feature_vals.append(feature_names[idx])

      results= {}
      for idx in range(len(feature_vals)):
          results[feature_vals[idx]]=score_vals[idx]
      
      return results

  def sort_coo(coo_matrix):
      tuples = zip(coo_matrix.col, coo_matrix.data)
      return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

  def extract_topn_from_vector(feature_names, sorted_items, topn=10):
      
      sorted_items = sorted_items[:topn]

      score_vals = []
      feature_vals = []

      for idx, score in sorted_items:
          fname = feature_names[idx]
          
          score_vals.append(round(score, 3))
          feature_vals.append(feature_names[idx])

      results= {}
      for idx in range(len(feature_vals)):
          results[feature_vals[idx]]=score_vals[idx]
      
      return results
  feature_names=cv.get_feature_names()

  tf_idf_vector=tfidf_transformer.transform(cv.transform([' '.join(text_n)]))

  sorted_items=sort_coo(tf_idf_vector.tocoo())

  keywords=extract_topn_from_vector(feature_names,sorted_items, 100)

  return keywords

In [ ]:
kw = []
i = 0
for data in tqdm(df_company_kw['text_for_keywords']):
  if i == 0:
    kw.append(get_kw(data, True))
  else:
    kw.append(get_kw(data, False))
  i += 1

In [ ]:
kw_normal = []
for keywords in kw:
  kw_ = []
  for k in keywords:
    kw_.append(k)
  kw_normal.append(kw_)

In [ ]:
df_company_kw['kw'] = [' '.join(x) for x in kw_normal]
df_company_kw['label'] = [encode_company_classes[x] for x in df_company_kw['label']]

In [ ]:
text_features = ['kw']

In [ ]:
X = df_company_kw.drop(['text_for_keywords', 'company_name', 'label'], axis=1)
y = df_company_kw['label']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y.values,test_size=0.2, shuffle=True)

In [ ]:
train_pool = Pool(
    X_train, 
    y_train, 
    text_features=text_features, 
    feature_names=list(X_train)
)
valid_pool = Pool(
    X_test, 
    y_test,
    text_features=text_features, 
    feature_names=list(X_train)
)

catboost_params = {
    'iterations': 3000,
    'learning_rate': 0.01,
    'eval_metric': 'Accuracy',
    'early_stopping_rounds': 2000,
    'use_best_model': True,
    'verbose': 100
}
model = CatBoostClassifier(**catboost_params)
model.fit(train_pool, eval_set=valid_pool)


In [ ]:
from sklearn.metrics import classification_report,accuracy_score
pred = model.predict(X_test)
print(classification_report(y_test,pred))
print(accuracy_score(y_test,pred))

In [ ]:
model.save_model('model_classif', format="cbm", export_parameters=None, pool=None)